# Проект: исследование уровня потребительской лояльности среди пользователей телекоммуникационной компании.

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 

Мне предстоит выгрузить данные в csv формате с помощью языка структурированных запросов (SQL) из базы данных, затем полчившийся датафрейм загрузить в Tableau и подготовить презентацию с дашбордами, описывающую участников опроса, а также попробовать дать рекомендации по улучшению сервиса для определенных групп пользователей.

## Подключение к базе

Подключим необходимые для работы библиотеки и загрузим базу данных с информацией об участниках проведенного опроса.

In [2]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [3]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## Выгрузка данных

Теперь с помощью SQL напишем запрос, который позволит нам составить датафрейм со всеми необходимыми данными об участниках опроса.

In [9]:
query = """
SELECT user_id,
    lt_day,
    CASE 
        WHEN lt_day <= 365 THEN 'Новый'
        ELSE 'Старый'
    END AS is_new,
    age,
    CASE
        WHEN gender_segment = 1 THEN 'Женский'
        ELSE 'Мужской'
    END AS gender_segment,
    os_name,
    cpe_type_name,
    country,
    city,
    SUBSTR(ag.title, 4) AS age_segment,
    SUBSTR(tr.title, 4) AS traffic_segment,
    SUBSTR(lt.title, 4) AS lifetime_segment,
    nps_score,
    CASE 
        WHEN nps_score >= 9 THEN 'сторонники'
        WHEN nps_score >= 7 THEN 'нейтралы'
        WHEN nps_score <= 6 THEN 'критики'
    END AS nps_group
    
FROM user AS u
LEFT JOIN location AS loc ON loc.location_id=u.location_id
LEFT JOIN age_segment AS ag ON ag.age_gr_id=u.age_gr_id
LEFT JOIN traffic_segment AS tr ON tr.tr_gr_id=u.tr_gr_id
LEFT JOIN lifetime_segment AS lt ON lt.lt_gr_id=u.lt_gr_id
"""

Теперь выведем первые 15 строк датафрейма, чтобы убедиться в правильности получившихся данных.

In [10]:
df = pd.read_sql(query, engine)
df.head(15)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женский,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,Старый,53.0,Мужской,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,Старый,57.0,Мужской,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники
3,A004TB,4190,Старый,44.0,Женский,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,сторонники
4,A004XT,1163,Старый,24.0,Мужской,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,сторонники
5,A005O0,5501,Старый,42.0,Женский,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,критики
6,A0061R,1236,Старый,45.0,Мужской,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,сторонники
7,A009KS,313,Новый,35.0,Мужской,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,сторонники
8,A00AES,3238,Старый,36.0,Женский,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,сторонники
9,A00F70,4479,Старый,54.0,Женский,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,сторонники


Данные выгрузились корректно, далее получившийся датафрейм преобразуем в CSV-файл и далее уже будем работать с ним локально в Tableau.

In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

Ниже прикрепляю ссылку на мою презентацию исследования в Tableau.

https://public.tableau.com/app/profile/ignat.pisar/viz/Tableauproject2_17228658160450/sheet0_2?publish=yes
